In [ ]:
"""
Kaggle Submission Notebook for CSIRO Biomass Prediction
========================================================
This notebook runs inference using trained models and creates submission.csv

Usage:
1. Train models locally: python train.py --config configs/exp/exp001.yaml
2. Upload model weights and code to Kaggle (run cell-2 locally)
3. Upload this notebook to Kaggle with the data sources attached
4. Run this notebook on Kaggle
"""

import os
import sys

# Kaggle環境ではコードDatasetからインポートするためにsys.pathを設定
# Dataset名: csiro-biomass-codes-{EXP_NAME} (例: csiro-biomass-codes-001)
if os.getenv("KAGGLE_DATA_PROXY_TOKEN"):
    sys.path.insert(0, "/kaggle/input/csiro-biomass-codes-015")

import config
import pandas as pd

# Check environment
print(f"Running in Kaggle: {config.IS_KAGGLE_ENV}")
print(f"Input directory: {config.INPUT_DIR}")
print(f"Output directory: {config.OUTPUT_DIR}")
print(f"Artifact directory: {config.ARTIFACT_DIR}")


In [ ]:
# # Run inference (SVR)
# from inference_svr import kaggle_inference

# # Configuration - Change experiment_dir to match your SVR experiment
# EXPERIMENT_DIR = "20251228_XXXXXX_exp014_dinov3_svr"  # <- ここを変更
# FOLDS = None  # None = use all 5 folds (0-4), or specify list like [0, 1, 2]
# IMG_SIZE = 960  # Image size (must be divisible by 16)

# # Run inference and create submission
# # TTA (4 flip variants) is always enabled
# submission_df = kaggle_inference(
#     experiment_dir=EXPERIMENT_DIR,
#     folds=FOLDS,
#     img_size=IMG_SIZE,
# )

# # Save submission
# submission_df.to_csv("submission.csv", index=False)
# print("Submission saved to submission.csv")
# print(f"Submission shape: {submission_df.shape}")
# submission_df.head(10)


In [ ]:
# Upload artifacts to Kaggle (run this locally after training)
# Uncomment and run this cell to upload trained models and code

# ============================================================
# アップロードする experiment_dir を指定
# data/output/014/1/ 内のディレクトリを確認して指定してください
# ============================================================
EXPERIMENT_DIR = "20251231_225539_exp015_dinov3_svr"  # ← ここを変更

if not config.IS_KAGGLE_ENV:
    import shutil

    from src.kaggle_utils.customhub import dataset_upload, model_upload

    experiment_path = config.OUTPUT_DIR / EXPERIMENT_DIR
    if not experiment_path.exists():
        raise FileNotFoundError(f"Experiment directory not found: {experiment_path}")

    print(f"Uploading experiment: {experiment_path}")

    # Copy backbone.pth to experiment directory for upload
    # backbone.pth is saved at OUTPUT_DIR/backbone.pth during feature extraction
    backbone_src = config.OUTPUT_DIR / "backbone.pth"
    backbone_dst = experiment_path / "backbone.pth"
    if backbone_src.exists() and not backbone_dst.exists():
        shutil.copy2(backbone_src, backbone_dst)
        print(f"Copied backbone.pth to {backbone_dst}")
    elif backbone_dst.exists():
        print(f"backbone.pth already exists at {backbone_dst}")
    else:
        print(f"WARNING: backbone.pth not found at {backbone_src}")

    # モデル重みを Kaggle Model としてアップロード
    # experiment_dir 内の全 run_name をアップロード
    model_upload(
        handle=config.ARTIFACTS_HANDLE,
        local_model_dir=experiment_path,
        update=True,  # Set to True to update existing model
    )

    # 実験コードを Kaggle Dataset としてアップロード
    # experiments/014/ ディレクトリのみ（プロジェクト全体ではない）
    dataset_upload(
        handle=config.CODES_HANDLE,
        local_dataset_dir=config.EXP_DIR,  # experiments/014/ のみ
        update=True,
    )
